In [1]:
import pandas as pd
from scipy.sparse import csr_matrix, diags
import numpy as np

directory = '/home/lorenzo/Scrivania/My_projects/DeepWalk/P2Vec/dataset/'

# Con questo siamo a posto... devo solo aggiungere poi una descrizione dettagliata

In [8]:
# load the dataset
name = 'orkut'

with open(directory + 'labels_' + name + '.txt') as f:
    lines = f.readlines()
    
    

df = pd.read_csv(directory + name + '.txt', skiprows = 4, header = None, sep = "\t")
df.rename(columns={0: "idx1", 1: "idx2"}, inplace = True)

# keep only the clusters with a size above a threshold
min_cluster_size = 20
assignment = []
for l in lines:
    a = l.split("\n")[0].split("\t")
    if len(a) > min_cluster_size:
        assignment.append([int(x) for x in a])

# assign each node to the largest cluster they belong to
label_dict = dict()
for i, a in enumerate(assignment):
    for j in a:
        if j not in label_dict.keys():
            label_dict[j] = [i]
        else:
            label_dict[j].append(i)

sizes = np.array([len(a) for a in assignment])

for x in label_dict.keys():
    label_dict[x] = label_dict[x][np.argmax(sizes[label_dict[x]])]

# sample down the network to the nodes actually appearing
all_nodes = list(label_dict.keys())
df = df[np.isin(df.idx1, all_nodes)]
df = df[np.isin(df.idx2, all_nodes)]

all_nodes = np.unique(np.concatenate([df.idx1, df.idx2]))
n = len(all_nodes)
mapper = dict(zip(all_nodes, np.arange(n)))

# obtain the label vector
ℓtrue = np.zeros(n)
for x in all_nodes:
    ℓtrue[mapper[x]] = label_dict[x]

EL = df.values

idx1 = [mapper[a] for a in EL[:,0]]
idx2 = [mapper[a] for a in EL[:,1]]

df = pd.DataFrame(columns = ['id1', 'id2'])
df.id1 = np.concatenate([idx1, idx2])
df.id2 = np.concatenate([idx2, idx1])

#------------------------------------------

df.to_csv(directory + name + '.csv')
    
label_df = pd.DataFrame(columns = ['node', 'label'])
label_df.node = np.arange(n)
label_df.label = ℓtrue
label_df.set_index('node', inplace = True)


label_df.to_csv(directory + name + '_label.csv')